# Importing libraries and classes

In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor as SKLearnRandomForestRegressor

from RandomForestRegressor import RandomForestRegressor
from LinearRegression import LogisticRegression

# Reading and splitting the data

In [10]:
df = pd.read_csv('./NSO_Population_Sex_dataset/NSO_POPULATION_DATA_CLEANED.csv')

feature_cols = ['District', 'Sex', 'Year', 'Population_Growth_Rate','Average_Population']
X = pd.get_dummies(df[feature_cols], columns=['District', 'Sex'])
y = df["Population"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# SKLearn Decision Trees

In [11]:
# To be added by Nathan Camilleri

# Custom Decision Trees

In [12]:
# To be added by Nathan Camilleri

# Random Forest Regressor using SKLearn
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [13]:
SKLearn_Random_Forest_Regressor = SKLearnRandomForestRegressor() # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

SKLearn_Random_Forest_Regressor.fit(X_train, y_train)

SKLearn_Random_Forest_Regressor_Prediction = SKLearn_Random_Forest_Regressor.predict(X_test)

SKLearn_Random_Forest_Regressor_Prediction_MAE = mean_absolute_error(y_test, SKLearn_Random_Forest_Regressor_Prediction)
SKLearn_Random_Forest_Regressor_Prediction_MSE = mean_squared_error(y_test, SKLearn_Random_Forest_Regressor_Prediction)

print(f"SKLearn Random Forest Regressor Mean Absolute Error: {SKLearn_Random_Forest_Regressor_Prediction_MAE}")
print(f"SKLearn Random Forest Regressor Mean Squared Error: {SKLearn_Random_Forest_Regressor_Prediction_MSE}")

SKLearn_Random_Forest_Regressor_Results_DF = pd.DataFrame({'Actual': y_test, 'Predicted': SKLearn_Random_Forest_Regressor_Prediction})
print(f"\nSKLearn Random Forest Predictions:\n{SKLearn_Random_Forest_Regressor_Results_DF}")


SKLearn Random Forest Regressor Mean Absolute Error: 0.0062180408329399256
SKLearn Random Forest Regressor Mean Squared Error: 0.00011546572724854736

SKLearn Random Forest Predictions:
       Actual  Predicted
97   0.173876   0.175979
39   0.609447   0.630738
83   0.206309   0.211902
143  0.364016   0.334013
45   0.898457   0.861194
57   0.687152   0.663145
163  0.005896   0.005963
124  0.203838   0.200466
165  0.006809   0.006221
71   0.241120   0.230947
87   0.225957   0.227473
167  0.007185   0.006390
177  0.000175   0.001336
121  0.194719   0.191163
148  0.208270   0.210033
138  0.273432   0.282884
119  0.190878   0.189319
24   0.330334   0.330390
162  0.005385   0.005496
90   0.241455   0.255110
147  0.199997   0.208606
151  0.227354   0.228045
41   0.689139   0.667539
21   0.330979   0.330262
35   0.589261   0.589833
171  0.011577   0.009963
155  0.295094   0.288854
112  0.182256   0.184448
129  0.185560   0.191301
33   0.584279   0.587139
4    0.332201   0.330855
38   0.590779 

# Custom Random Forest Regressor using SKLearn Decision Trees

In [15]:
SKLearn_Decision_Trees_Random_Forest_Regressor = RandomForestRegressor(n_estimators=100, max_depth=None, custom=False)
SKLearn_Decision_Trees_Random_Forest_Regressor.fit(X_train, y_train)

SKLearn_Decision_Trees_Random_Forest_Regressor_Prediction = SKLearn_Decision_Trees_Random_Forest_Regressor.predict(X_test)

SKLearn_Decision_Trees_Random_Forest_Regressor_MAE = mean_absolute_error(y_test, SKLearn_Decision_Trees_Random_Forest_Regressor_Prediction)
SKLearn_Decision_Trees_Random_Forest_Regressor_MSE = mean_squared_error(y_test, SKLearn_Decision_Trees_Random_Forest_Regressor_Prediction)

print(f"Custom Random Forest Regressor using SKLearn Decision Trees Mean Absolute Error: {SKLearn_Decision_Trees_Random_Forest_Regressor_MAE}")
print(f"Custom Random Forest Regressor using SKLearn Decision Trees Mean Squared Error: {SKLearn_Decision_Trees_Random_Forest_Regressor_MSE}")

SKLearn_Decision_Trees_Random_Forest_Regressor_Results_DF = pd.DataFrame({'Actual': y_test, 'Predicted': SKLearn_Decision_Trees_Random_Forest_Regressor_Prediction})
print(f"\nCustom Random Forest Regressor using SKLearn Decision Trees Predictions:\n{SKLearn_Decision_Trees_Random_Forest_Regressor_Results_DF}")

Custom Random Forest Regressor using SKLearn Decision Trees Mean Absolute Error: 0.006545388241443899
Custom Random Forest Regressor using SKLearn Decision Trees Mean Squared Error: 0.0001378993860993491

Custom Random Forest Regressor using SKLearn Decision Trees Predictions:
       Actual  Predicted
97   0.173876   0.176702
39   0.609447   0.632010
83   0.206309   0.208449
143  0.364016   0.336221
45   0.898457   0.855523
57   0.687152   0.657861
163  0.005896   0.005874
124  0.203838   0.201315
165  0.006809   0.006166
71   0.241120   0.229905
87   0.225957   0.227494
167  0.007185   0.006489
177  0.000175   0.001281
121  0.194719   0.192120
148  0.208270   0.212038
138  0.273432   0.279165
119  0.190878   0.188310
24   0.330334   0.330341
162  0.005385   0.005587
90   0.241455   0.253697
147  0.199997   0.201949
151  0.227354   0.229205
41   0.689139   0.661543
21   0.330979   0.329946
35   0.589261   0.590408
171  0.011577   0.009695
155  0.295094   0.285815
112  0.182256   0.1867

# Custom Random Forest Regressor

In [16]:
Custom_Random_Forest_Regressor = RandomForestRegressor(n_estimators=100, max_depth=100)
Custom_Random_Forest_Regressor.fit(X_train, y_train)

Custom_Random_Forest_Regressor_Prediction = Custom_Random_Forest_Regressor.predict(X_test)

Custom_Random_Forest_Regressor_MAE = mean_absolute_error(y_test, Custom_Random_Forest_Regressor_Prediction)
Custom_Random_Forest_Regressor_MSE = mean_squared_error(y_test, Custom_Random_Forest_Regressor_Prediction)

print(f"Custom Random Forest Regressor Mean Absolute Error: {Custom_Random_Forest_Regressor_MAE}")
print(f"Custom Random Forest Regressor Mean Squared Error: {Custom_Random_Forest_Regressor_MSE}")

Custom_Random_Forest_Regressor_Results_DF = pd.DataFrame({'Actual': y_test, 'Predicted': Custom_Random_Forest_Regressor_Prediction})
print(f"\nCustom Random Forest Regressor Predictions:\n{Custom_Random_Forest_Regressor_Results_DF}")

Custom Random Forest Regressor Mean Absolute Error: 0.00867481824076553
Custom Random Forest Regressor Mean Squared Error: 0.0002476557459798626

Custom Random Forest Regressor Predictions:
       Actual  Predicted
97   0.173876   0.178141
39   0.609447   0.637959
83   0.206309   0.214267
143  0.364016   0.312248
45   0.898457   0.904814
57   0.687152   0.734539
163  0.005896   0.005905
124  0.203838   0.204640
165  0.006809   0.006560
71   0.241120   0.244118
87   0.225957   0.233147
167  0.007185   0.006897
177  0.000175   0.001543
121  0.194719   0.192764
148  0.208270   0.212804
138  0.273432   0.286175
119  0.190878   0.192340
24   0.330334   0.330815
162  0.005385   0.005690
90   0.241455   0.264110
147  0.199997   0.208238
151  0.227354   0.233499
41   0.689139   0.735160
21   0.330979   0.329191
35   0.589261   0.592964
171  0.011577   0.011927
155  0.295094   0.304569
112  0.182256   0.184573
129  0.185560   0.188395
33   0.584279   0.589035
4    0.332201   0.330887
38   0.590

# Custom Linear Regression

In [17]:
# Normalisation of data 

X_train_normalized = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test_normalized = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)

In [18]:
Custom_Linear_Regression = LogisticRegression(learning_rate=0.01, num_iterations=1000)
Custom_Linear_Regression.fit(X_train_normalized, y_train)

Custom_Linear_Regression_Prediction = Custom_Linear_Regression.predict(X_test_normalized)

Custom_Linear_Regression_MAE = mean_absolute_error(y_test, Custom_Linear_Regression_Prediction)
Custom_Linear_Regression_MSE = mean_squared_error(y_test, Custom_Linear_Regression_Prediction)

print(f"Custom Linear Regression Mean Absolute Error: {Custom_Linear_Regression_MAE}")
print(f"Custom Linear Regression Mean Squared Error: {Custom_Linear_Regression_MSE}")


Custom_Linear_Regression_Results_DF = pd.DataFrame({'Actual': y_test, 'Predicted': Custom_Linear_Regression_Prediction})
print(f"\nCustom Linear Regression Predictions:\n{Custom_Linear_Regression_Results_DF}")

Custom Linear Regression Mean Absolute Error: 0.051118264798900755
Custom Linear Regression Mean Squared Error: 0.0041117131127046

Custom Linear Regression Predictions:
       Actual  Predicted
97   0.173876   0.181460
39   0.609447   0.745078
83   0.206309   0.227235
143  0.364016   0.340073
45   0.898457   0.784951
57   0.687152   0.749665
163  0.005896   0.066294
124  0.203838   0.243121
165  0.006809   0.072046
71   0.241120   0.264435
87   0.225957   0.255376
167  0.007185   0.077600
177  0.000175   0.037521
121  0.194719   0.222734
148  0.208270   0.260648
138  0.273432   0.295971
119  0.190878   0.209654
24   0.330334   0.355642
162  0.005385   0.064268
90   0.241455   0.277795
147  0.199997   0.253191
151  0.227354   0.283881
41   0.689139   0.758213
21   0.330979   0.329989
35   0.589261   0.715628
171  0.011577   0.087223
155  0.295094   0.314569
112  0.182256   0.172789
129  0.185560   0.230520
33   0.584279   0.700012
4    0.332201   0.332232
38   0.590779   0.738579
84   